In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np
import keras
from skimage.filters import gabor_kernel
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
# if K.backend()=='tensorflow':
#     K.set_image_dim_ordering("tf")
from skimage.color import rgb2gray
from scipy import ndimage as ndi
 
# Import Tensorflow with multiprocessing
import tensorflow as tf
import multiprocessing as mp
 
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
import pickle

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle

from keras.models import Sequential
from keras.layers import Conv2D
import os

#print(os.listdir("Images/"))

SIZE = 512 #Resize images

train_images = []

for directory_path in glob.glob("Images/"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        train_images.append(img)
        #train_labels.append(label)


        
train_images = np.array(train_images)

train_masks = [] 
for directory_path in glob.glob("Masks"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE, SIZE))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        #train_labels.append(label)


        
train_masks = np.array(train_masks)

X_train = train_images
y_train = train_masks
X_train = np.expand_dims(X_train, axis=3)
y_train = np.expand_dims(y_train, axis=3)

In [4]:
train_images.shape, train_masks.shape

((24, 512, 512), (24, 512, 512))

In [5]:
X_train.shape, y_train.shape

((24, 512, 512, 1), (24, 512, 512, 1))

In [6]:
def custom_gabor(shape, dtype=None):
    pi = np.pi
    orientation_spread = np.array([0, pi/4, pi/2, pi*3/4, pi, pi*5/4, pi*3/2, 2*pi])
    scales = np.linspace(2, 3, 6)
    real_kernels = []
    img_kernels = []
#     size, sigma, theta, lambda, gamma aspect ratio
    for orientation in orientation_spread:
        for scale in scales:
            real_kernel = cv2.getGaborKernel((5, 5), 3, orientation, scale, 1, 0)
            imaginary_kernel = cv2.getGaborKernel((5, 5), 3, orientation, scale, 1, np.pi / 2)
            real_kernels.append(real_kernel)
            img_kernels.append(imaginary_kernel)
    stacked_list = np.vstack((real_kernels, img_kernels))
    # stack number equal to number of color channel RGB: ([stacked_list, stacked_list, stacked_list])
    stacked_list = np.array([stacked_list])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    print(stacked_list.shape)

    stacked_list = K.variable(stacked_list)
    random = K.random_normal(shape, dtype=dtype)
    return stacked_list

In [7]:
def base_model(shape):
    model = Sequential()
    model.add(Conv2D(96, (5, 5), padding='same',kernel_initializer=custom_gabor, input_shape=shape))
    model.add(Activation('relu'))
    model.add(Conv2D(48, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [10]:
X_train.shape, y_train.shape, len(np.unique(y_train))

((24, 512, 512, 1), (20, 512, 512, 1), 2)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=20)

In [10]:
x_train.shape, y_train.shape

((20, 512, 512, 1), (20, 512, 512, 1))

In [11]:
%%time
model = base_model(x_train.shape[1:])
model.layers[0].trainable = False
model.summary()
history = model.fit(x_train, y_train, batch_size=9, epochs=1, validation_data=(x_test, y_test), shuffle=True)

(5, 5, 1, 96)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 512, 512, 96)      2496      
_________________________________________________________________
activation (Activation)      (None, 512, 512, 96)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 510, 510, 48)      41520     
_________________________________________________________________
activation_1 (Activation)    (None, 510, 510, 48)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 255, 255, 48)      0         
_________________________________________________________________
dropout (Dropout)            (None, 255, 255, 48)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 255, 2

ValueError: in user code:

    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /anaconda/envs/py37_default/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /anaconda/envs/py37_default/lib/python3

# Only Feature Extraction

In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle

from keras.models import Sequential
#from keras.layers import Conv2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import os

#print(os.listdir("Images/"))

SIZE = 512 #Resize images

train_images = []

for directory_path in glob.glob("Images/"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        train_images.append(img)
        #train_labels.append(label)


        
train_images = np.array(train_images)

train_masks = [] 
for directory_path in glob.glob("Masks"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE, SIZE))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        #train_labels.append(label)


        
train_masks = np.array(train_masks)

X_train = train_images
Y_train = train_masks

X_train = np.expand_dims(X_train, axis=3)
Y_train = np.expand_dims(Y_train, axis=3)



In [1]:

from keras import backend as K

def custom_gabor(shape, dtype=None):
    pi = np.pi
    orientation_spread = np.array([0, pi/4, pi/2, pi*3/4, pi, pi*5/4, pi*3/2, 2*pi])
    scales = np.linspace(2, 3, 6)
    real_kernels = []
    img_kernels = []
#     size, sigma, theta, lambda, gamma aspect ratio
    for orientation in orientation_spread:
        for scale in scales:
            real_kernel = cv2.getGaborKernel((5, 5), 3, orientation, scale, 1, 0)
            imaginary_kernel = cv2.getGaborKernel((5, 5), 3, orientation, scale, 1, np.pi / 2)
            real_kernels.append(real_kernel)
            img_kernels.append(imaginary_kernel)
    stacked_list = np.vstack((real_kernels, img_kernels))
    # stack number equal to number of color channel RGB: ([stacked_list, stacked_list, stacked_list])
    stacked_list = np.array([stacked_list])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    print(stacked_list.shape)

    stacked_list = K.variable(stacked_list)
    random = K.random_normal(shape, dtype=dtype)
    return stacked_list

In [12]:
X_train.shape, Y_train.shape

((24, 512, 512, 1), (24, 512, 512, 1))

In [26]:
model = Sequential()
model.add(Conv2D(96, (5, 5), padding='same',kernel_initializer=custom_gabor, input_shape=(SIZE, SIZE, 1)))
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding='same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))

model.summary()

(5, 5, 1, 96)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 512, 512, 96)      2496      
_________________________________________________________________
activation_9 (Activation)    (None, 512, 512, 96)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 512, 512, 48)      41520     
_________________________________________________________________
activation_10 (Activation)   (None, 512, 512, 48)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512, 512, 48)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 512, 512, 64)      27712     
_________________________________________________________________
activation_11 (Activation)   (None, 512,

In [13]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.15, random_state=20)

In [14]:
# x_train.shape, y_train.shape

((20, 512, 512, 1), (20, 512, 512, 1))

In [31]:
X = model.predict(X_train)



In [32]:
X.shape

(24, 512, 512, 64)

In [33]:

X = X.reshape(-1, X.shape[3])

Y = Y_train.reshape(-1)

print(X.shape, Y.shape)

(6291456, 64) (6291456,)


In [34]:

dataset = pd.DataFrame(X)
dataset['Label'] = Y

print(dataset['Label'].unique())
print(dataset['Label'].value_counts())


# dataset = dataset[dataset['Label'] != 0]


X_for_RF = dataset.drop(labels = ['Label'], axis=1)
Y_for_RF = dataset['Label']

[0 1]
1    3197813
0    3093643
Name: Label, dtype: int64


# Random Forest Classifier

In [35]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_for_RF, Y_for_RF, test_size=0.15, random_state=20)

In [ ]:
%%time
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 250, max_depth = 32, random_state = 42)

# Train the model on training data
# Ravel Y to pass 1d array instead of column vector
model.fit(X_train, y_train) #For sklearn no one hot encoding


In [ ]:
from sklearn.metrics import accuracy_score

prediction = model.predict(X_test)
##Check accuracy on test dataset.  
print ("Accuracy = ", accuracy_score(y_test, prediction)) 

In [ ]:
import joblib

# Save to file in the current working directory
joblib_file = "GABOR-CNN-RF.pkl"
joblib.dump(model, joblib_file)


